In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))


import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats
from scipy.special import inv_boxcox
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from xgboost import XGBRegressor

import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt


In [ ]:
train = pd.read_csv("../input/train.tsv", sep='\t', index_col=0)
test = pd.read_csv("../input/test.tsv", sep='\t', index_col=0)

In [ ]:
train.isnull().sum(), test.isnull().sum()
train.shape, test.shape

In [ ]:
train.price.describe()
_,ax = plt.subplots(figsize=(6,6))
sns.distplot(train.price, hist=False, ax=ax)

The distribution of 'price' is highly skewed, so we might have to log transform this for applying regression models. If we use tree based models we can skip this transformation as tree models are quite robust to skewness and outliers. 

In [ ]:
_,ax = plt.subplots(figsize=(6,6))
_ = stats.probplot(train.price, dist=stats.norm, plot=ax)
print ("Skewness : {0}".format(stats.skew(train.price)))

The distribution of price is highly skewed towards positive side. We will transform our price using stats.boxcox . We add 1 to the price to accumulate items with 0 price.

In [ ]:
price_box, lmbda = stats.boxcox(train['price']+1)
print ("Lambda : {0}".format(lmbda))
_,ax = plt.subplots(figsize=(5,5))
_ = stats.probplot(price_box, dist=stats.norm, plot=ax)
print ("Skewness : {0}".format(stats.skew(price_box)))

Ohh here we have a negative lambda, which might create troubles during our inverse coxbox transformation. So lets try square root or logarithmic transformations and see which turns out to be better. 


In [ ]:
price_log = np.log(train['price']+1)
_,ax = plt.subplots(figsize=(5,5))
_ = stats.probplot(price_log, dist=stats.norm, plot=ax)
print ("Skewness : {0}".format(stats.skew(price_log)))

In [ ]:
price_sqrt = np.sqrt(train['price'])
_,ax = plt.subplots(figsize=(5,5))
_ = stats.probplot(price_sqrt, dist=stats.norm, plot=ax)
print ("Skewness : {0}".format(stats.skew(price_sqrt)))

Logarithmic transformation has less skewness and thus closer to normal.

In [ ]:
train['price'] = np.log1p(train['price'])

In [ ]:
all_data = pd.concat([train.drop(['price'], axis=1), test])
all_data.shape

In [ ]:
def cat_no(category) : 
    try : return len([s.strip() for s in category.split('/')])
    except : return 0

all_data['cat_no'] = all_data.category_name.apply(cat_no)
all_data.cat_no.value_counts()

In category_name as we see there are  multiple subcategories which can be 3,4 or 5. There are null values as well. for which cat_no is 0. So we create different features for each subcategory. 

In [ ]:
def split_cat(category) : 
    try: 
        cat_no = len([s.strip() for s in category.split('/')])
        if (cat_no==3):
            return category.split('/')+['NA', 'NA']
        elif (cat_no==4):
            return category.split('/')+['NA']
        elif (cat_no==5):
            return category.split('/')
    except : 
        return ['NA', 'NA', 'NA', 'NA', 'NA']

In [ ]:
all_data['cat1'], all_data['cat2'], all_data['cat3'], all_data['cat4'], all_data['cat5'] = \
zip(*all_data['category_name'].apply(split_cat))
all_data.head()

In [ ]:
all_data.loc[all_data.cat4=='NA'].shape[0]/all_data.shape[0], all_data.loc[all_data.cat5=='NA'].shape[0]/all_data.shape[0]

More than 99% values in cat4 and cat5 are 'NA', so we can drop these 2 from our model. 
Now we can map our categorical columns to numbers. 

In [ ]:
all_data['cat1'] = all_data['cat1'].astype('category').cat.codes
all_data['cat2'] = all_data['cat2'].astype('category').cat.codes
all_data['cat3'] = all_data['cat3'].astype('category').cat.codes
all_data['cat4'] = all_data['cat4'].astype('category').cat.codes
all_data['cat5'] = all_data['cat5'].astype('category').cat.codes

In [ ]:
all_data.head()

Getting back our train and test from all_data

In [ ]:
train_df = all_data[:train.shape[0]]
train_df['price'] = train.price
test_df = all_data[train.shape[0]:]
train_df.shape, test_df.shape

In [ ]:
train.isnull().sum(), test.isnull().sum()

In train we have null values in category_name, brand_name and item_description. In test there are null values in brand_name and category_name. 
1. We will fill null values only in brand_name feature. 
2. For null in category_name we already have cat1, cat2 cat3 as 'NA'
3. We drop item_description from our model so no need to fill null here. 


In [ ]:
train_df.loc[:, ['brand_name']], test_df.loc[:,['brand_name']] = (df.loc[:,['brand_name']].fillna('NA') for df in [train_df, test_df])
train_df['brand_name'] = train_df['brand_name'].astype('category').cat.codes
test_df['brand_name'] = test_df['brand_name'].astype('category').cat.codes

Now a little bit of intuition can help to identify that linear regression won't be of help here. So we experiment with tree based models as the data can be divided into various regions based on various predictors(first on category, then on brand, shipping, condition etc) to get a good prediction. Also we chose to ignore the 'name'( as it is represented by 'category' and 'brand') and item_description(which is highly subjective and need a NLP approach to be used).

Also we are using original train without adjusting for skewness as tree based models are quite robust to skewness and outliers.

In [ ]:
predictors = ['item_condition_id', 'brand_name', 'shipping', 'cat_no', 'cat1', 'cat2', 'cat3']
X = train_df[predictors]
y = train_df.price
_=sns.distplot(train_df.price, hist=False)
train_df.price.describe()

Our training set contains 1.4 million samples and thus tuning our gradient boosting model will take a logn time. So let's downsample our train_df so that training and tuning are fast and when all parameters are decided we can train on full model. 

In [ ]:
small_train = train_df.sample(frac=0.1, random_state=11)
small_train.shape
small_train.price.describe()
_=sns.distplot(small_train.price, hist=False)

The sample we have taken has almost same stats for price feature and a similar distribuion. 

In [ ]:
# Root Mean square Logarithmic error
def rmsle(y_true, y_pred):
    return np.sqrt(np.square( np.log(y_pred+1)- np.log(y_true+1) ).sum() / y_true.shape[0])

# Root mean square logarithmic error
def scorer(estimator, X, y):
    estimator.fit(X, y)
    y_pred = estimator.predict(X)
    return -1*np.sqrt(np.square( np.log(y_pred+1)- np.log(y+1) ).sum() / y.shape[0])

In [ ]:
def model_fit_evaluate(estimator, train, predictors, performCV=True, featureImportances=True, cv_folds=5):
    # fit the estimator on train
    estimator.fit(train[predictors], train['price'])
    
    # predict train
    train_pred = estimator.predict(train[predictors])
    
    # perform CV
    if performCV : 
        cv_score = cross_val_score(estimator, train[predictors], train['price'], cv=cv_folds, scoring=scorer)
    
    # print RMSLE cv score : 
    print ("Mean : {0}\nStd : {1}".format(cv_score.mean(), cv_score.std()))

    #feature importances
    if featureImportances : 
        feat_imp = pd.Series(estimator.feature_importances_, predictors).sort_values(ascending=False)
        feat_imp.plot(kind='bar', title='Feature Importance')
        plt.ylabel('Feature imortance score')
    
    
    

Let's first create a baseline model with default parameters.

In [ ]:
gbr0 = GradientBoostingRegressor(random_state=11, verbose=1)
predictors = ['item_condition_id', 'brand_name', 'shipping', 'cat_no', 'cat1', 'cat2', 'cat3']
model_fit_evaluate(gbr0, small_train, predictors)

This is the RMSLE for a base model and we will try to get better reults by tuning parameters. 

In [ ]:
params1 = {'n_estimators':range(20,81,10)}
gbr1 = GradientBoostingRegressor(learning_rate=0.2, subsample=0.8, min_samples_split=2000, 
                                 min_samples_leaf=100, max_depth=20, random_state=11, 
                                 verbose=1, max_features='sqrt')
gsearch1 = GridSearchCV(gbr1, params1, scoring=scorer, cv=5, verbose=1, return_train_score=True)
gsearch1.fit(small_train[predictors], small_train.price)

In [ ]:
gsearch1.cv_results_['mean_test_score']
pd.DataFrame(gsearch1.cv_results_['mean_test_score'], index=range(20, 81, 10))

As we see when n_estimators is 80 we have maximum cv_score or minimum RMLSE. It will be better if we try to increase learning rate and look for an optimum value of n_estimators in this range(ie under 100), but for now lets continue with 80. 

In [ ]:
params2 = {'n_estimators':range(100,301,50)}
gbr2 = GradientBoostingRegressor(learning_rate=1.5, subsample=0.8, min_samples_split=1500, 
                                 min_samples_leaf=100, max_depth=2, random_state=11, 
                                 verbose=1, max_features='sqrt' )
gsearch2 = GridSearchCV(gbr2, params2, cv=5, verbose=1, return_train_score=True)
gsearch2.fit(small_train[predictors], small_train.price)

In [ ]:
params3 = {'max_depth':range(5,16,2), 'min_samples_split':range(200,1001,200)}
gbr3 = GradientBoostingRegressor(n_estimators=80, subsample=0.8, 
                                 min_samples_leaf=100, random_state=11, 
                                 verbose=1, max_features='sqrt' )
gsearch3 = GridSearchCV(gbr3, params3, cv=5, verbose=1, return_train_score=True)
gsearch3.fit(small_train[predictors], small_train.price)

In [ ]:
# Root Mean square Logarithmic error
def rmsle(y_true, y_pred):
    return np.sqrt(np.square( np.log(y_pred+1)- np.log(y_true+1) ).sum() / y_true.shape[0])

# Root mean square logarithmic error
def scorer(estimator, X, y):
    estimator.fit(X, y)
    y_pred = estimator.predict(X)
    return -1*np.sqrt(np.square( np.log(y_pred+1)- np.log(y+1) ).sum() / y.shape[0])

In [ ]:
predictors = ['item_condition_id', 'brand_name', 'shipping', 'cat_no', 'cat1', 'cat2', 'cat3']
X = train_df[predictors]
y = train_df.price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=11, shuffle=True)

In [ ]:
rfr0 = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=11, verbose=1, max_depth=10)
score = cross_val_score(rfr0, X, y, scoring=scorer, verbose=1, cv=5)
score.mean()

After trying various values of max_depth and n_estimators, we found that the best rmsle is obtained by n_estimators=100 and max_depth = None. So now lets just try to increase n_estimators so improve score. We won;t use cross_val_score as it tkes lot of time, instead check score on a test set. 

In [ ]:
rfr1 = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=11, verbose=2, max_depth=20)
rfr1.fit(X_train, y_train)
y_pred = rfr1.predict(X_test)
rmsl_error = rmsle(y_test, y_pred)
print ("RMSLE : {0}".format(rmsl_error))

Increasing n_estimators doesn;t give better results but changing max_depth slightly improves rmlse. So now lets train our random forest on full data with these parameters and submit the result. 

In [ ]:
rfr2 = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=11, verbose=2, max_depth=20)
rfr2.fit(X, y)
y_pred = rfr2.predict(test_df[predictors])
y_pred = inv_boxcox(y_pred, lmbda) - 1

In [ ]:
sub = pd.DataFrame(y_pred, index=test.index, columns=['price'])
sub.to_csv('sub_rfr2.csv')

Now we use Gradient Boosting model to preidct prices. We use smaller n_estimators to reduce trainig time, and also since n_estimators is correlated with learning rate which we are chossing using cross validation. 

In [ ]:
gbr = GradientBoostingRegressor(learning_rate=1, n_estimators=500, max_depth=8, subsample=0.8, random_state=11, verbose=1, max_features='sqrt')
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
y_pred = inv_boxcox(y_pred, lmbda) - 1
y_true = inv_boxcox(y_test, lmbda) - 1
rmsl_error = rmsle(y_true, y_pred)
print ("RMSLE : {0}".format(rmsl_error))

In [ ]:
gbr = GradientBoostingRegressor(learning_rate=1, n_estimators=300, max_depth=8, max_features='sqrt', subsample=0.8, random_state=11, verbose=1)
gbr.fit(X, y)

In [ ]:
y_pred = gbr.predict(test_df[predictors])
y_pred = np.expm1(y_pred)


In [ ]:
sub = pd.DataFrame(y_pred, index=test.index, columns=['price'])
sub.loc[sub['price']<0]=0
sub.to_csv('sub_gbr2.csv')

In [1]:
gbr = GradientBoostingRegressor(max_features='sqrt', n_estimators=500, learning_rate=0.1, verbose=1)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
y_pred = inv_boxcox(y_pred, lmbda) - 1
rmsl_error = rmsle(inv_boxcox(y_test, lmbda) - 1, y_pred)
print ("Error  : {0}".format(rmsl_error))


y_pred = gbr.predict(test[['item_condition_id', 'category_no', 'brand_no', 'shipping']])
y_pred = inv_boxcox(y_pred, lmbda) - 1

In [ ]:
sub = pd.DataFrame(y_pred, index=test.index, columns=['price'])
sub.to_csv('sub_gbr.csv')